# Exploring Spatiotemporal Segregation Dynamics

In [ ]:
%load_ext watermark
%watermark -v -a "author: eli knaap" -d -u -p segregation,libpysal,geopandas,geosnap

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import hvplot.pandas

from segregation import singlegroup, multigroup, dynamics, batch
from geosnap import datasets, Community
from geosnap.analyze import segdyn

In [ ]:
from geosnap.visualize import plot_timeseries

In [ ]:
dc = gpd.read_parquet("data/dc_income.parquet")

In [ ]:
dc.head()

In [ ]:
from IPython.display import IFrame

## Income Segregation over Time

In [ ]:
plot_timeseries(dc, 'share_very_high_inc', nrows=2, ncols=4, figsize=(18,10), cmap='Blues', alpha=0.8)

### Temporal Dynamics

#### Multi-Group Indices

In [ ]:
cols = ['very_low_inc', 'low_inc', 'med_inc', 'high_inc', 'very_high_inc']

In [ ]:
multi_by_time = segdyn.multigroup_tempdyn(dc, cols)

In [ ]:
multi_by_time

In [ ]:
multi_by_time.T.plot()

In [ ]:
# removing the GlobalDistortion coef lets us see what's happening with the rest of the indices
multi_by_time.iloc[1:].T.plot()

Most indices are decreasing slightly over time

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10,4))

multi_by_time.loc['MultiDissim'].plot(ax=axs[0])
multi_by_time.loc['MultiDissim'].plot(kind='bar', ax=axs[1])

fig.suptitle("Multigroup Dissimilarity")

One that **isn't**, is SimpsonsConcentration, which is increasing over time. Another index that bucks the trend is SimpsonsInteraction, which is decreasing over time (corresponding with an increse in segregation). The divergence between indices tells us that segregation may be changing in different ways across its different dimensions.

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10,4))

multi_by_time.loc['SimpsonsConcentration'].plot(ax=axs[0])
multi_by_time.loc['SimpsonsConcentration'].plot(kind='bar', ax=axs[1])

fig.suptitle("Simpson's Concentration")

#### Single-Group Indices

In [ ]:
from geosnap.analyze.segdyn import singlegroup_tempdyn

In [ ]:
singlegroup_tempdyn?

In [ ]:
segs_single = segdyn.singlegroup_tempdyn(dc, group_pop_var='very_high_inc', total_pop_var='total', )

In [ ]:
segs_single

In [ ]:
segs_single.T.hvplot(height=600)

<https://www.jstor.org/stable/2579183>

In [ ]:
IFrame('https://www.jstor.org/stable/2579183', height=600, width=800)

In [ ]:
(segs_single.T[['Gini', 'Entropy', 'Dissim', 'Atkinson']].hvplot(title='Evenness Dimension', width=380, height=400).opts(legend_position='bottom', show_grid=True) +
segs_single.T[['AbsoluteConcentration', 'RelativeConcentration' , 'Delta']].hvplot(title='Concentration Dimension', width=380, height=400).opts(legend_position='bottom', show_grid=True) +
segs_single.T[['AbsoluteClustering', 'Isolation', 'CorrelationR', 'Interaction', 'SpatialProxProf']].hvplot(title='Exposure/Clustering Dimension', width=380, height=400).opts(legend_position='bottom', show_grid=True))


In [ ]:
segs_single.T[['AbsoluteClustering', 'Isolation', 'SpatialProxProf', 'Interaction']].pct_change(periods=5) # we should only compare non-overlapping intervals

Between the sampling periods 2008-2012 and 2013-2017:
- the isolation index increased by 5.2%
- the absolute clustering index increased by 12.4%.  
- the spatial proximity profile increased by 17.6%

Between the sampling periods 2009-2013 and 2014-2018:
- the isolation index increased by 7.9%
- the absolute clustering index increased by 18.2%
- the spatial proximity profile increased by 21.9%

## Space-Time Dynamics

In [ ]:
from segregation.singlegroup import Entropy

In [ ]:
d = segdyn.spacetime_dyn(dc, singlegroup.Entropy, group_pop_var='very_high_inc', total_pop_var='total', distances=list(range(500,5500,500)))

In [ ]:
d.plot(cmap='Reds')

Entropy is falling the fastest at small scales (the gap is wider on the left-hand side of the graph than the right-hand side)

In [ ]:
iso = segdyn.spacetime_dyn(dc, singlegroup.Isolation, group_pop_var='very_high_inc', total_pop_var='total', distances=list(range(500,5500,500)))

In [ ]:
iso.plot(cmap='Reds')

Isolation is growing the fastest at large scales (the gap is wider with larger distances on the right)

In [ ]:
from geosnap.visualize import animate_timeseries

In [ ]:
animate_timeseries(dc, 'share_very_high_inc', filename='figs/dc_high_inc_change.gif', fps=1.5)

In [ ]:
from IPython.display import Image

In [ ]:
Image('figs/dc_high_inc_change.gif',width=800)

The story in DC is one of increasing isolation by the affluent at large spatial scales. 

By many metrics, the region appears to be becoming *less* segregated by income over time, but that increase in evenness is largely due to the entire region getting richer. By contrast, looking at the exposure dimension shows that the residents with the highest incomes spend increasingly less time in environments with other income groups, and that change is happening quickest at large spatial scales. Put differently, we see a trend akin to agglomeration, whereby the large wealthy enclaves are becoming moreso

(remember this is example makes some *very* liberal assumptions about the input data, so the "takeaways" here are just for illustration)

## Using geosnap as a Dashboard Engine

The Python dashboarding ecosystem is evolving quickly, so we won't opine on which platform or toolset is best. But if you have a personal favorite, `geosnap` is performant to power an urban analytics dashboard on-the-fly. The example below wraps a simple streamlit interface around the workflow above that lets us explore *every* metro region quickly

**example**:  <https://github.com/knaaptime/incseg_app>

![](https://github.com/knaaptime/incseg_app/raw/main/data/incseg_app_demo.png)

## Demo